In [1]:
from DES_Parameters import *
from tqdm import tqdm

In [2]:
def xorstring(bit1, bit2, n): 
    ans = '' 
    for i in range(n):
        if(bit1[i] == bit2[i]):
            ans = ans + '0'
        else:
            ans = ans + '1'
    return ans

def splitter(block, n):
    return [ block[i:i + n] for i in range(0, len(block), n)]

def shift_left(block, n):
    return block[n:] + block[0:n]

def permute(block, permuter, n):
    out = ''
    for i in range(n):
        out = out + block[ permuter[i] - 1]
    return out

def read_file(loc):
    textpairs = []
    f = open(loc, "r")
    while True:
        text1 = f.readline()
        text2 = f.readline()
        if not text1:
            break
        textpairs.append( [text1[:-1],text2[:-1]] )
    return textpairs

In [3]:
def unpack(spacked):
    packed = [char for char in spacked] 
    binary = ''
    for i in range(len(packed)):
        binary = binary + charmap[packed[i]]
    return binary

def pack(sbinary):
    binary = splitter(sbinary, 4)
    packed = ''
    for i in range(len(binary)):
        packed = packed + bitmap[binary[i]]
    return packed

def unpack4(spacked):
    packed = [char for char in spacked] 
    binary = ''
    for i in range(len(packed)):
        binary = binary + tcharmap[packed[i]]
    return binary

def pack4(sbinary):
    binary = splitter(sbinary, 4)
    packed = ''
    for i in range(len(binary)):
        packed = packed + tbitmap[binary[i]]
    return packed

In [4]:
def keygen(keybits, rounds, permute_need = 1):
    if(permute_need == 1):
        keybits = permute(keybits, PC1, 56)
        
    rkeybit = []
    
    left = keybits[0:28]
    right = keybits[28:56]
    
    for i in range(rounds):
        left  = shift_left( left, shifts[i] )
        right = shift_left( right, shifts[i] )
        round_key = permute( left+right, PC2, 48)
        rkeybit.append( round_key )
    return rkeybit

In [5]:
def des_expand(block):
    return permute(block, E, 48)

def des_key(block, key):
    return xorstring(block, key, 48)

def des_si(block, boxno):
    row = int(block[0]) *2 + int(block[5])
    col = int(block[1]) *8 + int(block[2]) *4 + int(block[3]) *2 + int(block[4])
    val = bin( S[boxno-1][row][col] )[2:].zfill(4)
    return val

def des_Sbox(block):
    out = ''
    block = [block[i:i+6] for i in range(0, len(block), 6)]
    for i in range(8):
        out = out + des_si( block[i], i+1 )
    return out

def des_permute(block):
    return permute(block, P, 32)

In [6]:
def DES(pt, round_key, rounds):
    
    #INITIAL PERMUTATION
    pt    = permute(pt, IP, 64)
    left  = pt[:32]
    right = pt[32:]
    
    #ROUNDS STRUCTURE
    for i in range(rounds):
        r_expand = des_expand(right)
        r_keyed  = des_key(r_expand, round_key[i])
        r_S      = des_Sbox(r_keyed)
        r_per    = des_permute(r_S)
        r_out    = xorstring(r_per, left, 32)
        

        left, right = right, r_out
        
    # REVERSE FINAL PERMUTATION
    ct = permute(left+right, RFP, 64)
    return ct

# DES  Compute Parameters needed for 6 rounds

In [7]:
def get_all_param(ceq):
    
    input_xor = permute( unpack(ceq), IP_INV, 64)
    
    ex  = splitter( des_expand( unpack(ceq)[:32] ), 6)
    skbox = []
    for i in range(len(ex)):
        if(ex[i] == '000000'):
            skbox.append(i+1) 
            
    l5xor = unpack(ceq)[32:]
    return {
        'input_xor' : input_xor,
        'Sboxes'    : skbox,
        'L5xor'     : l5xor
    }

In [8]:
# Finding the xor for plain text input pair
chr1 = '4008000004000000'
chr2 = '0020000800000400'

In [9]:
c1par = get_all_param(chr1)
c2par = get_all_param(chr2)

In [10]:
c1par

{'input_xor': '0000000000000000100000000001000000000000000000000100000000000000',
 'Sboxes': [2, 5, 6, 7, 8],
 'L5xor': '00000100000000000000000000000000'}

In [11]:
c2par

{'input_xor': '0000000000000000000010000000000100000000000100000000000000000000',
 'Sboxes': [1, 2, 4, 5, 6],
 'L5xor': '00000000000000000000010000000000'}

# Search for 6th round Key

In [12]:
keyiset = []
for i in range(64):
    keyiset.append( str(bin(i)[2:].zfill(6)) )

In [13]:
def get_pairs(ct1, ct2, l5xor):
    t1 = permute( unpack4(ct1), RFP_INV, 64)
    t2 = permute( unpack4(ct2), RFP_INV, 64)
    l61, r61 = t1[:32], t1[32:]
    l62, r62 = t2[:32], t2[32:]
    
    l5xor = permute(l5xor, P_INV, 32)
    r6xor = permute(  xorstring(r61,r62,32) , P_INV, 32) 
    Fp = xorstring(l5xor, r6xor, 32)
    
    e1 = des_expand(l61)
    e2 = des_expand(l62)
    
    return (splitter(Fp, 4), splitter(e1, 6), splitter(e2, 6) )

In [14]:
def get_keys(ciphertexts, l5xor, sboxes):
    key_freq = {}
    for i in sboxes:
        key_freq[i] = {}
        for k in keyiset:
            key_freq[i][k] = 0
        
    
    for c in tqdm( range(len(ciphertexts)) ):
        ct = ciphertexts[c]
        postS, prekey1, prekey2 = get_pairs(ct[0], ct[1], l5xor)
        
        for i in sboxes:
            
            for k in keyiset:
                
                preS1 = des_si( xorstring(prekey1[i-1], k, 6) , i)
                preS2 = des_si( xorstring(prekey2[i-1], k, 6) , i)
                preS = xorstring( preS1, preS2, 4)
                
                if( preS == postS[i-1]):
                    key_freq[i][k] = key_freq[i][k] + 1
    
    for i in sboxes:
        key_freq[i] = sorted( key_freq[i].items(), key = lambda kv:(kv[1], kv[0]) , reverse=True)
    return key_freq    

In [16]:
ciphertext1 = read_file('texts/ct1_clean.txt')
ciphertext2 = read_file('texts/ct2_clean.txt')

In [17]:
f1 = get_keys(ciphertext1, c1par['L5xor'], c1par['Sboxes'] )
for i in f1.keys():
    print(i, '  ', f1[i][0])

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 624.76it/s]

2    ('110011', 80)
5    ('011011', 43)
6    ('110110', 82)
7    ('010101', 57)
8    ('110100', 44)


In [18]:
f2 = get_keys(ciphertext2, c2par['L5xor'], c2par['Sboxes'] )
for i in f2.keys():
    print(i, '  ', f2[i][0])

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 637.23it/s]

1    ('111101', 52)
2    ('110011', 48)
4    ('000111', 99)
5    ('011011', 49)
6    ('110110', 98)


In [19]:
key6 = '111101'+'110011'+'xxxxxx'+'000111'+'011011'+'110110'+'010101'+'110100'

In [20]:
key6

'111101110011xxxxxx000111011011110110010101110100'

#  Find Key Mapping

In [21]:
kbits = ''
for i in range(56):
    kbits = kbits + chr(i+1)

In [22]:
rkeybit = []
    
left = kbits[0:28]
right = kbits[28:56]
    
for i in range(6):
        
    left  = shift_left( left, shifts[i] )
    right = shift_left( right, shifts[i] )
        
    round_key = permute( left+right, PC2, 48)
        
    rkeybit.append( round_key )

In [23]:
smap = []
for x in rkeybit[5]:
    smap.append(ord(x))

In [24]:
tups = {}
for i in range(48):
    tups[smap[i]-1] = key6[i]

In [25]:
keymain = ''
for i in range(56):
    if i in tups.keys():
         keymain = keymain + tups[i]
    else:
        keymain = keymain + 'x'

In [26]:
print(keymain)

x11xx1xx01011x100xx11x11001x1010011x11011010x10x0101x011


# Brute Force Remaining bits

In [27]:
file = []
for i in range( 2**14 ):
    file.append( (bin(i)[2:].zfill(14)) )

In [28]:
def tempkeygen(key, block):
    counter = 0
    newkey = ''
    for i in range(56):
        if(key[i] == 'x' ):
            newkey = newkey + ( block[counter] )
            counter = counter + 1
        else:
            newkey = newkey + (key[i])
    return newkey

In [32]:
pt_final = unpack4('sfqrtsnnufpurmof')
ct_final = unpack4('omiuomrkkilmumfr')

for i in tqdm( range(2**14) ):
    
    tkey = tempkeygen(keymain, file[i])
    round_keys  = keygen(tkey, 6, permute_need = 0 )
    ctemp = DES( pt_final, round_keys, 6)
    if(ctemp == ct_final):
        print(file[i])
        final_key = (tkey)
        break

 21%|████████████████                                                           | 3520/16384 [00:00<00:02, 4951.77it/s]

00110111000000


In [35]:
# DES 56 bit KEY
final_key

'01101110010111100111101100101010011011011010010001010011'

# Decrypt the Password

In [36]:
password = 'fpfhqoljnnnhfpkhrshthpmppoglhoup'
round_keys = keygen( final_key, 6,  permute_need= 0 )
round_rev_keys = round_keys[::-1]

In [37]:
# DES Decrypt 
left  = pack4( DES( unpack4(password[:16]) , round_rev_keys,6 ))
right = pack4( DES( unpack4(password[16:]) , round_rev_keys,6 ))

In [40]:
decrypted_pass = left+right
decrypted_pass

'lumkmjmhljlkljlklrmlifififififif'

In [39]:
for x in splitter( unpack4(decrypted_pass), 8):
    print( chr(int(x,2)) ,end= '' )

outrdedelv000000